# Demo of the solarchallenge package

This file contains a demonstration of the solarchallenge. The purpose is to simulate the power generated by an orbiting solar panel. For the challenge, it is assumed that the solar photovoltaic cells are always pointing at zenith.

The package allows for performing orbit propagation with perturbation, assuming a two-body problem, computing the power generated using the Pindado-Cubas equation, and performing thermal analysis of the solar panel, using a lumped mass approach.

In [ ]:
from solarchallenge.solarpanel.solarpanel import SolarPanel
from solarchallenge.visualization.plotting import plot_temperature, plot_heat, plot_power, plot_orbit, plot_coe, plot_IV
from solarchallenge.model import Model
from solarchallenge.constants import (J2_EARTH, J3_EARTH, R_MEAN_EARTH, J2000,
                                      A_EARTH, ECC_EARTH, INC_EARTH, RAAN_EARTH,
                                      ARGP_EARTH, NU_EARTH, D)
from solarchallenge.solarpanel.utils import compute_angle_solar_incidence, compute_solar_radiance
from solarchallenge.trajectory.orbit import Orbit
from solarchallenge.trajectory.propagator import propagate_cowell
from solarchallenge.trajectory.perturbations import constant_accel_wrapper, J2_perturbation_wrapper, J3_perturbation_wrapper
from solarchallenge.bodies.bodies import Sun, Earth
import numpy as np

### Setting up an orbit and propagation

In [ ]:
# Setting up Earth Orbit at J2000 using Orbit class (EME2000 Frame)
earth_orbit = Orbit.from_coe(a=A_EARTH, ecc=ECC_EARTH, inc=INC_EARTH, raan=RAAN_EARTH,
                             argp=ARGP_EARTH, nu=NU_EARTH,
                             epoch=J2000,
                             attractor=Sun)

# Setting up a solar panel/satellite Orbit - ISS example (J2000 Frame)
r_iss = [859.07256, -4137.20368,  5295.56871] # Position [km]
v_iss = [7.37289205, 2.08223573, 0.43999979] # Velocity [km/s]

iss_orbit = Orbit.from_vector(r=r_iss, v=v_iss, epoch=J2000, attractor=Earth)
#iss_orbit = Orbit.from_coe(a=R_MEAN_EARTH+1000, ecc=0.25, inc=30, raan=15, argp=154, nu=255, epoch=J2000, attractor=Earth)

#Orbits are propagated using Cowell's propagation method

n_points = 10000
tf = 2*D.total_seconds()

t_eval = np.linspace(0, tf, n_points) # Time points to store solution
r_prop, v_prop = propagate_cowell(orbit=iss_orbit,
                                  tf=tf,
                                  t_eval=t_eval,
                                  perturbations=None)

r_prop = np.swapaxes(r_prop, 0, 1)
v_prop = np.swapaxes(v_prop, 0, 1)

time = iss_orbit.epoch.timestamp() + t_eval

plot_orbit(r_prop, R_MEAN_EARTH*0.5)
plot_coe(r_prop, v_prop, time, iss_orbit.attractor.mu)

In [ ]:
# Perturbations can also be included in the Propagation

perturbations = [constant_accel_wrapper(0E-7),
                 J2_perturbation_wrapper(J2=J2_EARTH, R=R_MEAN_EARTH)]

r_prop, v_prop = propagate_cowell(orbit=iss_orbit,
                                  tf=tf,
                                  t_eval=t_eval,
                                  perturbations=perturbations)

r_prop = np.swapaxes(r_prop, 0, 1)
v_prop = np.swapaxes(v_prop, 0, 1)

plot_orbit(r_prop, R_MEAN_EARTH*0.5)
plot_coe(r_prop, v_prop, time, iss_orbit.attractor.mu)

# Setting up a solar panel

The solar panel is set up using the class SolarPanel, which stores the solar panel information. This module was developed having in mind the computation of power to be as realistic as possible. For this effect, a method similar to the single diode model was integrated into this work. More information on this model can be found in https://oa.upm.es/67369/1/IEEETran_Aerospece_2021.pdf.

In [ ]:
#To initialize a SolarPanel object, we define a dictionary with the solar panel specifications, 
#that can be easily found in solar panel data sheets.

panel_information = {
    'a': 1,     # ideality factor of the diode
    'n_s': 1,   # Number of series connected cells
    'i_sc': 0.506, #short circuit current
    'v_oc': 2.667, #open circuit voltage
    'i_mp': 0.487, #current at maximum power
    'v_mp': 2.371, #voltage at maximum power
    'alpha_i_sc': 0.32/1000.0, #thermal coefficients
    'alpha_v_oc': -6.0/1000.0,
    'alpha_v_mp': -6.1/1000.0,
    'alpha_i_mp': 0.28/1000.0,
    'tref': 28,  # temperature of reference
    'solar_irr_ref': 1367, #solar irradiance of reference
    'eff': 1 # Panel efficiency (To account for other losses)
}

# Only used if the thermal model is activated, but required to initialize the object
additional_panel_information = {
     "area": 1,  # Area
     "mass": 1, # Mass
     "c": 1000, # Specific heat capacity
     "emittance_back": 1.0,
     "emittance_front": 1.0,
     "absorptance_back": 1.0,
     "absorptance_front": 1.0,
}

panel = SolarPanel.from_dict(panel_information | additional_panel_information)

In [ ]:
#The produced power depends on the solar_irradiation, angle of incidence, temperature and voltage of operation

solar_irr = 1367   #Depends on the Sun-panel distance
angle_incidence = 0
temperature = 28 #solar panel temperature
voltage = 2 # Voltage of operation

power, __, __ = panel.compute_power(solar_irr=solar_irr,
                                    angle_incidence=angle_incidence,
                                    temperature=temperature,
                                    voltage=voltage)

print(f"Power at v = {voltage}:", power)

#If voltage is None, the voltage of maximum power is used
power, __, __ = panel.compute_power(solar_irr=solar_irr,
                                    angle_incidence=angle_incidence,
                                    temperature=temperature,
                                    voltage=None)

print(f"Power at maximum voltage:", power)

In [ ]:
# Print of the IV plots

print("IV plots depending on temperature")
plot_IV(panel=panel, solar_irr=1367, temperature = np.array([20,40,60,80]))

print("IV plots depending on effective solar irradiance")
plot_IV(panel=panel, solar_irr=np.array([200, 600, 1000, 1367]), temperature = 28)

### Model class -  Propagating the orbit for a specific date interval.

The solarchallenge package has the class Model, used to store the propagation, power and temperature information, but also to be used as the driver of the solar panel simulation.

In [ ]:
#Create a new object of type Model
simulator = Model()

#Specify the orbits used in the simulation (The propagation is performed using the Model class)
simulator.set_orbit_body(earth_orbit)
simulator.set_orbit_solar_panel(iss_orbit)

#Specify Perturbation
perturbations_panel = [constant_accel_wrapper(0E-7), J2_perturbation_wrapper(J2=J2_EARTH, R=R_MEAN_EARTH)]
perturbations_planet = []

#Propagate the orbit for a given interval
#The propagation of the solar panel and planet starts at the orbit state epoch.
#After propagation, the position and velocity state vectors of both solar panel and planet are stored.

simulator.propagate_orbit(start=J2000 + 28 * D, # The datetime for start argument needs to be later than the epochs of the orbits
                          end=J2000 + 29 * D,
                          npoints=10000, 
                          perturbations_panel = perturbations_panel,
                          perturbations_planet = perturbations_planet)

plot_orbit(simulator.r_panel, R_MEAN_EARTH * 0.5)
plot_coe(simulator.r_panel, simulator.v_panel, simulator.time, simulator.orbit_panel.attractor.mu)

### Model class - Computing power for propagated trajectory

After orbit propagation, the power provided by the solar panel can be computed.

For the challenge, we assume the solar panel to always be pointing to zenith. Therefore, all the required angles can be retrieved from the position of the planet with respect to the Sun, and the position of the solar panel with respect to Earth.

The solar irradiance is retrieved as a function of the distance between the sun and the solar panel.

In [ ]:
#Set solar panel characteristics
simulator.set_solar_panel(panel)

#Compute power for interval of time
# If maximum_power is set to False, the power computation is perfomed using the defined voltage of operation
simulator.compute_power(temperature=28, voltage=2, maximum_power=False)

#Plot power and effective irradiance
solar_irr = compute_solar_radiance(simulator.r_planet, simulator.r_panel)
theta = compute_angle_solar_incidence(simulator.r_planet, simulator.r_panel)

plot_power(time=simulator.time, power=simulator.power, solar_irr=solar_irr, theta=theta)

In [ ]:
# To use the maximum power voltage, we can set the maximum_power=True
# It ignores the value of the voltage argument

#Compute power for interval of time
simulator.compute_power(temperature=28, voltage=2, maximum_power=True)

#Plot power and effective irradiance
solar_irr = compute_solar_radiance(simulator.r_planet, simulator.r_panel)
theta = compute_angle_solar_incidence(simulator.r_planet, simulator.r_panel)

plot_power(time=simulator.time, power=simulator.power, solar_irr=solar_irr, theta=theta)

### Model class - Thermal analysis and power computation

The power computation is dependent on the temperature of the solar panel. Thus, a thermal analysis can be performed by the package, assuming a lumped mass approach. In this approach, the heat conduction inside the temperature is assumed to be instantaneous, i.e. the temperature is uniformly distributed.

The evolution of temperature with respect to time depends on the area, absorptance and emittance of the solar panel surfaces, mass and specific heat capacity of the panel structure, and the heat exchange between the and space.

The radiation from deep space impacting the solar panel are the solar irradiance, Earth albedo and Earth emitted Infrared.

The satellite releases energy through electric production (power) and black-body radiation (Stefan-Boltzmann)

In [ ]:
# To perform thermal analysis, we set the thermal_model boolean to True
# If it's set to True, the temperature argument is ignored

#The thermal analysis is performed as.
# 1) Determination of the steady state solution to retrieve initial temperature (q_in = q_out)
# 2) Forward Euler method to compute Temperature using the orbit propagation time

simulator.compute_power(temperature=28, voltage=2, maximum_power=True, thermal_model=True)

plot_temperature(time=simulator.time, temperature=simulator.temperature)
plot_heat(time=simulator.time, q_earth_ir=simulator.q_earth_ir, q_earth_alb=simulator.q_earth_alb,
          q_solar_front=simulator.q_solar_front, q_solar_back=simulator.q_solar_back,
          q_sat_rad=simulator.q_sat_rad)
plot_power(time=simulator.time, power=simulator.power, solar_irr=solar_irr, theta=theta)

# Code example

The power computation is dependent on the temperature of the solar panel. Thus, a thermal analysis can be performed by the package, assuming a lumped mass approach. In this approach, the heat conduction inside the temperature is assumed to be instantaneous, i.e. the temperature is uniformly distributed.

The evolution of temperature with respect to time depends on the area, absorptance and emittance of the solar panel surfaces, mass and specific heat capacity of the panel structure, and the heat exchange between the and space.

The radiation from deep space impacting the solar panel are the solar irradiance, Earth albedo and Earth emitted Infrared.

The satellite releases energy through electric production (power) and black-body radiation (Stefan-Boltzmann)


In [ ]:
### Setting orbits

panel_orbit = Orbit.from_vector(r = [-2384.46, 4729.01, 3050.46],
                          v = [-7.36138, -2.98997, 1.64354],
                          epoch = J2000,
                          attractor = Earth)

panel_perturbations = [constant_accel_wrapper(1E-6),
                 J2_perturbation_wrapper(J2=J2_EARTH, R=R_MEAN_EARTH)]

earth_orbit = Orbit.from_coe(a=A_EARTH, ecc=ECC_EARTH, inc=INC_EARTH, raan=RAAN_EARTH, argp=ARGP_EARTH, nu=NU_EARTH,
                             epoch=J2000,
                             attractor=Sun)

earth_perturbations = []

### Setting solar panel

panel_information = {
    'a': 1,     # ideality factor of the diode
    'n_s': 1,   # Number of series connected cells
    'i_sc': 0.506, #short circuit current
    'v_oc': 2.667, #open circuit voltage
    'i_mp': 0.487, #current at maximum power
    'v_mp': 2.371, #voltage at maximum power
    'alpha_i_sc': 0.32/1000.0, #thermal coefficients
    'alpha_v_oc': -6.0/1000.0,
    'alpha_v_mp': -6.1/1000.0,
    'alpha_i_mp': 0.28/1000.0,
    'tref': 28,  # temperature of reference
    'solar_irr_ref': 1367, #solar irradiance of reference
    'eff': 1 # Panel efficiency (To account for other losses)
}

# Only used if the thermal model is activated, but required to initialize the object
additional_panel_information = {
     "area": 1,  # Area
     "mass": 1, # Mass
     "c": 1000, # Specific heat capacity
     "emittance_back": 1.0,
     "emittance_front": 1.0,
     "absorptance_back": 1.0,
     "absorptance_front": 1.0,
}

panel = SolarPanel.from_dict(panel_information | additional_panel_information)

### Simulation driver
simulator = Model()

simulator.set_orbit_body(earth_orbit)
simulator.set_orbit_solar_panel(panel_orbit)
simulator.set_solar_panel(panel)

#Propagate the orbit for a given interval
simulator.propagate_orbit(start=J2000 + 28 * D, # The datetime for start argument needs to be later than the epochs of the orbits
                          end=J2000 + 29 * D,
                          npoints=10000, 
                          perturbations_panel = panel_perturbations,
                          perturbations_planet = earth_perturbations)

#Compute power
simulator.compute_power(temperature=28, voltage=2, maximum_power=True, thermal_model=True)

### Plots
plot_orbit(simulator.r_panel, R_MEAN_EARTH * 0.5)

plot_coe(simulator.r_panel, simulator.v_panel, simulator.time, simulator.orbit_panel.attractor.mu)

plot_temperature(time=simulator.time, temperature=simulator.temperature)

plot_heat(time=simulator.time, q_earth_ir=simulator.q_earth_ir, q_earth_alb=simulator.q_earth_alb,
          q_solar_front=simulator.q_solar_front, q_solar_back=simulator.q_solar_back,
          q_sat_rad=simulator.q_sat_rad)

solar_irr = compute_solar_radiance(simulator.r_planet, simulator.r_panel)
theta = compute_angle_solar_incidence(simulator.r_planet, simulator.r_panel)
plot_power(time=simulator.time, power=simulator.power, solar_irr=solar_irr, theta=theta)